In [4]:
pip install langchain-community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [5]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

··········


In [6]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [7]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "/content/414759-1-_5_Nike-NPS-Combo_Form-10-K_WR.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

106


In [ ]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

FORM 10-KFORM 10-K

{'producer': 'Wdesk Fidelity Content Translations Version 008.001.016', 'creator': 'Workiva', 'creationdate': '2023-07-20T22:09:22+00:00', 'author': 'anonymous', 'moddate': '2023-07-26T15:13:52+08:00', 'title': 'Nike 2023 Proxy', 'source': '/content/414759-1-_5_Nike-NPS-Combo_Form-10-K_WR.pdf', 'total_pages': 106, 'page': 0, 'page_label': '1'}


In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

501

In [13]:
pip install -qU langchain-mistralai

In [19]:
import getpass
import os

if not os.environ.get("MISTRALAI_API_KEY"):
  os.environ["MISTRALAI_API_KEY"] = getpass.getpass("Enter API key for MistralAI: ")

from langchain_mistralai import MistralAIEmbeddings
os.environ["HF_TOKEN"] = "your_token"
embeddings = MistralAIEmbeddings(
    mistral_api_key="your token",
    model="mistral-embed"
)

In [20]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 1024

[-0.031341552734375, 0.0247802734375, 0.017730712890625, -0.00920867919921875, 0.033905029296875, 0.043304443359375, 0.0216522216796875, 0.01126861572265625, -0.012542724609375, -0.0093536376953125]


In [21]:
pip install -qU langchain-core

In [22]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [23]:
ids = vector_store.add_documents(documents=all_splits)

In [24]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='in the United States:
U.S. RETAIL STORES NUMBER
NIKE Brand factory stores  213 
NIKE Brand in-line stores (including employee-only stores)  74 
Converse stores (including factory stores)  82 
TOTAL  369 
In the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further information.
NIKE, INC.       2' metadata={'producer': 'Wdesk Fidelity Content Translations Version 008.001.016', 'creator': 'Workiva', 'creationdate': '2023-07-20T22:09:22+00:00', 'author': 'anonymous', 'moddate': '2023-07-26T15:13:52+08:00', 'title': 'Nike 2023 Proxy', 'source': '/content/414759-1-_5_Nike-NPS-Combo_Form-10-K_WR.pdf', 'total_pages': 106, 'page': 5, 'page_label': '6', 'start_index': 3218}


In [25]:
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

page_content='PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this 
"Annual Report"), the terms "we," "us," "our," "NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries 
and affiliates, collectively, unless the context indicates otherwise.
Our principal business activity is the design, development and worldwide marketing and selling of athletic footwear, apparel, 
equipment, accessories and services. NIKE is the largest seller of athletic footwear and apparel in the world. We sell our products 
through NIKE Direct operations, which are comprised of both NIKE-owned retail stores and sales through our digital platforms 
(also referred to as "NIKE Brand Digital"), to retail accounts and to a mix of independent distributors, licensees and sales 
representatives in nearly all countries around the world. We also offer interactive consumer services and experiences t

In [26]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.8784244923521904

page_content='FISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTS
The following tables present NIKE Brand revenues disaggregated by reportable operating segment, distribution channel and 
major product line:
FISCAL 2023 COMPARED TO FISCAL 2022
• NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported 
and currency-neutral basis, respectively. The increase was due to higher revenues in North America, Europe, Middle East & 
Africa ("EMEA"), APLA and Greater China, which contributed approximately 7, 6, 2 and 1 percentage points to NIKE, Inc. 
Revenues, respectively. 
• NIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues, increased 10% and 16% on a reported and 
currency-neutral basis, respectively. This increase was primarily due to higher revenues in Men's, the Jordan Brand, 
Women's and Kids' which grew 17%, 35%,11% and 10%, respectively, on a wholesale equivalent basis.' metadata={'prod

In [27]:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='(Dollars in millions) 2023 2022
Net income $ 5,070 $ 6,046 
Add: Interest expense (income), net  (6)  205 
Add: Income tax expense  1,131  605 
Earnings before interest and taxes $ 6,195 $ 6,856 
EBIT Margin: Calculated as total NIKE, Inc. EBIT divided by total NIKE, Inc. Revenues. Our EBIT Margin calculation for fiscal 
2023 and fiscal 2022 is as follows:
YEAR ENDED MAY 31,
(Dollars in millions) 2023 2022
Numerator
Earnings before interest and taxes $ 6,195 $ 6,856 
Denominator
Total NIKE, Inc. Revenues $ 51,217 $ 46,710 
EBIT Margin  12.1 %  14.7 %
2023 FORM 10-K   29' metadata={'producer': 'Wdesk Fidelity Content Translations Version 008.001.016', 'creator': 'Workiva', 'creationdate': '2023-07-20T22:09:22+00:00', 'author': 'anonymous', 'moddate': '2023-07-26T15:13:52+08:00', 'title': 'Nike 2023 Proxy', 'source': '/content/414759-1-_5_Nike-NPS-Combo_Form-10-K_WR.pdf', 'total_pages': 106, 'page': 32, 'page_label': '33', 'start_index': 3238}
